In [2]:
import pandas as pd
import requests
import selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 
from bs4 import BeautifulSoup
from time import sleep 
import re
import sys 
import os
sys.path.append(os.path.abspath("../src"))   


import funciones as f

In [22]:
# for mercado in lista_supers:
#     driver = webdriver.Chrome()
#     url_mercados = f"https://super.facua.org/{mercado}/"
#     driver.get(url_mercados)
#     sleep(3)
#     lista_num_producto=[]
    
#     for #aquí quiero meter los productos usando beutiful
#         for num_producto in range(1, len())
#             driver.find_element("css selector", f"body > section:nth-child(4) > div > div.row.gx-4.gx-lg-5.row-cols-2.row-cols-md-3.row-cols-xl-4.justify-content-center > div:nth-child({num_producto}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
#             sleep(2)
#             lista_variante_producto=[]
#             for #aqui quiero meter las variantes de producto usando beutiful
#                 for num_variante in range(1, len()):
#                     driver.find_element("css selector", f"body > section:nth-child(4) > div > div:nth-child(7) > div:nth-child({num_variante}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
#                 #usar BeautifulSoup para coger la primera tabla y que me la guarde
#                 url_facua = f"https://super.facua.org/{mercadona}/{producto}/{variante}/"
#                 res_facua = requests.get(url_facua)
#                 sopa_mercados = BeautifulSoup(res_facua.content, "html.parser")
#                 lista_mercados = sopa_mercados.findAll("a", {"class" :"nav-link"})


In [16]:
# driver = webdriver.Chrome()
# url_mercados = f"https://super.facua.org/mercadona/"
# driver.get(url_mercados)
# #quitar cookie
# driver.find_element("css selector", "#rcc-confirm-button").click()
# sleep(2)
# url_variante=driver.find_element("css selector", "body > section:nth-child(4) > div > div:nth-child(7) > div:nth-child(1) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a")
# driver.get(url_variante)
# res_variante = requests.get(url_variante)
# sopa_variantes = BeautifulSoup(res_variante.content, "html.parser")
# lista_mercados = sopa_mercados.findAll("table")
# lista_mercados

In [ ]:
def sacar_tabla(tabla_indiv):
    tr_tabla=tabla_indiv.findAll("tr")
    th_tabla = tabla_indiv.findAll("th")
    td_tabla=tabla_indiv.findAll("td")
    df_cuerpo=pd.DataFrame([r.findAll('td') for r in tr_tabla][1:]).map(lambda x: x.text)
    dicc_encabezado=dict(enumerate(pd.DataFrame(th_tabla).T.values[0]))  #enumerate porque coincide el 0,1 y 2 con los titulos de las columnas
    df_tabla= df_cuerpo.rename(dicc_encabezado, axis=1)
    return df_tabla

Sacar lista supers

In [23]:
url_super = "https://super.facua.org/"
res_facua = requests.get(url_super)
sopa_mercados = BeautifulSoup(res_facua.content, "html.parser")
lista_mercados = sopa_mercados.findAll("a", {"class" :"nav-link"})
lista_supers= [mercado.getText().lower() for mercado in lista_mercados]
lista_supers.pop()
lista_supers


['mercadona', 'alcampo', 'carrefour', 'hipercor', 'dia', 'eroski']

In [19]:
for mercado in lista_supers:
    df_tablas= pd.DataFrame()
    driver = webdriver.Chrome()
    url_mercados = f"https://super.facua.org/{mercado}/"
    driver.get(url_mercados)
    sleep(3)
    #aceptar cookie
    driver.find_element("css selector", "#rcc-confirm-button").click()
    #hacer aquí la lista de productos
    res_producto = requests.get(url_mercados)
    sopa_productos = BeautifulSoup(res_producto.content, "html.parser")
    lista_prod = sopa_productos.findAll("p", {"class" :"fw-bolder"})
    lista_productos= [prod.getText().lower().replace(" ", "-") for prod in lista_prod]

    for i, producto in enumerate(lista_productos):
        driver.execute_script("window.scrollBy(0, 500);")
        sleep(2)
        #selecciona producto
        driver.find_element("css selector", f"body > section:nth-child(4) > div > div.row.gx-4.gx-lg-5.row-cols-2.row-cols-md-3.row-cols-xl-4.justify-content-center > div:nth-child({i}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
        url_producto = driver.page_source
        #hacer aquí la lista de variantes
        res_variante= requests.get(url_producto)
        sopa_variante = BeautifulSoup(res_variante.content, "html.parser")
        lista_var = sopa_variante.findAll("p", {"class" :"fw-bolder"})
        lista_variantes= [variante.getText().lower().replace(" ", "-").replace(",", "").replace("º","").replace(".","") for variante in lista_var]

        for j ,variante in enumerate(lista_variantes):
            driver.execute_script("window.scrollBy(0, 500);")
            sleep(2)
            #selecciona variante
            driver.find_element("css selector", f"body > section:nth-child(4) > div > div:nth-child(7) > div:nth-child({j}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
            url_variante = driver.page_source
            res_facua = requests.get(url_variante)
            sopa_mercados = BeautifulSoup(res_facua.content, "html.parser")
            tabla_general= sopa_mercados.findAll("table")
            tablaindiv=tabla_general[0]
            #por aquí crear las columnas necesarias usando lista de productos y variante (mercado se pone formateado tal cual)
            df_tablas=pd.concat([df_tablas, sacar_tabla(tablaindiv)],  ),
    
    df_tablas.to_csv(f'datos/datos{mercado}.csv', index=False)


Sacar lista productos

In [28]:
url_mercados = f"https://super.facua.org/mercadona/"
res_producto = requests.get(url_mercados)
sopa_productos = BeautifulSoup(res_producto.content, "html.parser")
lista_prod = sopa_productos.findAll("p", {"class" :"fw-bolder"})
lista_productos= [mercado.getText().lower().replace(" ", "-") for mercado in lista_prod]
for i, producto in enumerate(lista_productos):
    id_prod= i
    nombre_producto= producto
 
 

Aquí creamos la lista de tuplas para añadir las columnas que queremos que hay que multiplicar por la len de la tabla de historico para que ocupe toda la columna

In [32]:
tupla= (mercado, producto, variante)
lista=[tupla for i in range (1,len(tablaindiv))]
lista

[('texti', 'agdh', 'uu'),
 ('texti', 'agdh', 'uu'),
 ('texti', 'agdh', 'uu'),
 ('texti', 'agdh', 'uu')]

In [30]:
nombre_producto

'leche'

Sacar lista variantes


In [42]:
url_variante = "https://super.facua.org/mercadona/aceite-de-girasol/"
res_variante= requests.get(url_variante)
sopa_variante = BeautifulSoup(res_variante.content, "html.parser")
lista_var = sopa_variante.findAll("p", {"class" :"fw-bolder"})
lista_variantes= [variante.getText().lower().replace(" ", "-").replace(",", "").replace("º","").replace(".","") for variante in lista_var]
for i, variante in enumerate(lista_variantes):
    id_var= i
    nombre_variante= variante

In [43]:
nombre_variante

'aceite-de-girasol-refinado-02-hacendado-5-l'

In [21]:
sacar_tabla(tablaindiv)

,Día,Precio (€),Variación
0,12/07/2024,"1,45",=
1,13/07/2024,"1,45",=
2,14/07/2024,"1,45",=
3,15/07/2024,"1,45",=
4,16/07/2024,"1,45",=
...,...,...,...
101,21/10/2024,"1,48",=
102,22/10/2024,"1,48",=
103,23/10/2024,"1,48",=
104,24/10/2024,"1,48",=


In [4]:
tabla=tabla[0]

In [ ]:
tr_tabla=tabla.findAll("tr")
th_tabla = tabla.findAll("th")
td_tabla=tabla.findAll("td")

In [16]:
df_cuerpo=pd.DataFrame([r.findAll('td') for r in tr_tabla][1:]).map(lambda x: x.text)
dicc_encabezado=dict(enumerate(pd.DataFrame(th_tabla).T.values[0]))  #enumerate porque coincide el 0,1 y 2 con los titulos de las columnas
df_tabla= df_cuerpo.rename(dicc_encabezado, axis=1)
df_tabla

,Día,Precio (€),Variación
0,12/07/2024,"1,45",=
1,13/07/2024,"1,45",=
2,14/07/2024,"1,45",=
3,15/07/2024,"1,45",=
4,16/07/2024,"1,45",=
...,...,...,...
101,21/10/2024,"1,48",=
102,22/10/2024,"1,48",=
103,23/10/2024,"1,48",=
104,24/10/2024,"1,48",=


In [12]:
dict(enumerate(pd.DataFrame(enunciados).T.values[0]))


{0: 'Día', 1: 'Precio (€)', 2: 'Variación'}